In [10]:
# import libraries.
import numpy as np
from sklearn.utils import shuffle
import tensorflow as tf
import datetime
# from tensorflow import keras
# from keras.models import Sequential
# from keras.layers import Dense
# from keras import layers
# from keras.wrappers.scikit_learn import KerasRegressor
# # import keras
# from keras.layers import BatchNormalization
# from keras import optimizers
# from keras import regularizers

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
# import keras
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
#import sounddevice as sd
# import tensorflow as tf
# tf.enable_eager_execution()
from pystoi.stoi import stoi
import h5py
######################
#import libraries.
import matplotlib.pyplot as plt
from tabulate import tabulate
import time
import os
import librosa
from librosa.core import stft, istft
####import sounddevice as sd
import time
print('imported')
# #######################
# Data_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'
Data_path = os.getcwd()
# input_tr_name = 'lilftr_refrmd_3h'
# input_val_name = 'ftr_refrmd_test'
# target_tr_name = 'lilsingle_dataset_log_16'
# target_val_name = 'clean_data_wav_norm'
tfrecord_folder_parent = 'tfrecord_files'
tfrecord_folder = 'tfrecord_files_10h_norm'
tfrecord_val_folder = 'validation_10h_norm'
ckpt_folder = '3'
# input_tr_path = os.path.normpath(os.path.join(Data_path,input_tr_name))
# input_val_path = os.path.normpath(os.path.join(Data_path,input_val_name))
# target_tr_path = os.path.normpath(os.path.join(Data_path,target_tr_name))
# target_val_path = os.path.normpath(os.path.join(Data_path,target_val_name))

# file = h5py.File(target_val_path+'.hdf5','r')
# val_y = file[target_val_name][0:]
# file.close()
# file = h5py.File(input_val_path+'.hdf5','r')
# val_x = file[input_val_name][0:]
# file.close()

# hh = h5py.File(target_tr_path+'.hdf5', 'r')
# d=hh[target_tr_name]
# len_data=d.shape
# hh.close()
# len_data = (684108, 257)
len_data = (100000, 257)
val_len = (97278,257)
# file = h5py.File(target_val_path+'.hdf5','r')
# val_y = file[target_val_name]
# val_len=val_y.shape
# file.close()

w=3
# print(len_data)
#######################
#define reconstruct function to reconstruct sound from framed signal.
def reconstruct(wave,angle):
    recon = np.sqrt(np.power(10, wave))
    recon1 = recon*np.cos(angle)+recon*np.sin(angle)*1j
    recon = librosa.core.istft((recon1.T), hop_length=200, win_length=500, window='hann')
    return recon
#######################
I=0
global batch_size
batch_size = 128
# epochs_num=50
global datalen
datalen=len_data[0]
########################
# visible = w*len_data[1]
# hidden = h[0]
# visible1 = h[0]
# hidden1 = h[1]
# visible2 = h[1]
# hidden2 = len_data[1]

# layer1 = rbm_layer(visible, hidden, 20, 128, 0.0005, [np.eye(visible)], [np.zeros((1,visible))], 1, len_data[0], name1, name2)
# layer2 = rbm_layer(visible1, hidden1, 20, 128, 0.0005, [np.eye(visible),layer1[0]], [np.zeros((1,visible)),layer1[2]], 2, len_data[0], name1, name2)
# layer3 = rbm_layer(visible2, hidden2, 20, 128, 0.0005, [np.eye(visible),layer1[0],layer2[0]], [np.zeros((1,visible)),layer1[2],layer2[2]], 3, len_data[0], name1, name2)
# layer4 = rbm_layer(visible3, hidden3, 20, 128, 0.0005, [np.eye(visible),layer1[0],layer2[0],layer3[0]], [np.zeros((1,visible)),layer1[2],layer2[2],layer3[2]], 4, len_data[0], name1, name2)
# rbm_layers=[layer1,layer2,layer3,layer4]
# h5f = h5py.File('rbm_params.h5', 'w')
# h5f.create_dataset('rbm_params', data=rbm_layers)
# h5f.close()
# np.savetxt('rbm_layers.txt',rbm_layers)
# l10=np.loadtxt('layers10.txt')
# l11=np.loadtxt('layers11.txt')
# l12=np.loadtxt('layers12.txt')
# l20=np.loadtxt('layers20.txt')
# l21=np.loadtxt('layers21.txt')
# l22=np.loadtxt('layers22.txt')
# l30=np.loadtxt('layers30.txt')
# l31=np.loadtxt('layers31.txt')
# l32=np.loadtxt('layers32.txt')
# l40=np.loadtxt('layers40.txt')
# l41=np.loadtxt('layers41.txt')
# l42=np.loadtxt('layers42.txt')
###############################
# file.close()
h = [512,512]
seed = 7
rate1 = 0.1
rate2 = 0.2
from tensorflow.keras.layers import Activation
# from keras.layers import Activation
np.random.seed(seed)
model = Sequential()
act1 = layers.LeakyReLU(alpha=0.1)
model.add(layers.Dropout(rate1, noise_shape=None, seed=None))
# ,kernel_regularizer=regularizers.l2(0.001)
model.add(Dense(h[0], input_dim = w*len_data[1]))
model.add(BatchNormalization())
model.add(act1)
# model.add(Activation('sigmoid'))
act2=layers.LeakyReLU(alpha=0.1)
model.add(layers.Dropout(rate2, noise_shape=None, seed=None))
model.add(Dense(h[1]))
model.add(act2)
# act3=layers.LeakyReLU(alpha=0.1)
# # model.add(layers.Dropout(rate, noise_shape=None, seed=None))
# model.add(Dense(h[2]))
# model.add(act3)
act=layers.LeakyReLU(alpha=0.01)
model.add(Dense(len_data[1]))
#############################################
import os
from natsort import natsorted

def _parse_function(example_proto):
    features = {"X": tf.FixedLenFeature((3*257), tf.float32),
              "Y": tf.FixedLenFeature((257), tf.float32)}
    parsed_features = tf.parse_single_example(example_proto, features)
    print("i was here")
    return parsed_features["X"], parsed_features["Y"]

# orig_path = os.getcwd()
tfrecord_path = os.path.normpath(os.path.join(Data_path,tfrecord_folder_parent,tfrecord_folder))
sorted_names = natsorted(os.listdir(tfrecord_path))
trainfilenames = []
for i in sorted_names:
    trainfilenames.append(os.path.normpath(os.path.join(tfrecord_path,i)))
filenames = tf.placeholder(tf.string, shape=[None])
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(_parse_function)  # Parse the record into tensors.
dataset = dataset.repeat()  # Repeat the input indefinitely.
dataset = dataset.batch(batch_size)
iterator = dataset.make_initializable_iterator()


tfrecord_path_val = os.path.normpath(os.path.join(Data_path,tfrecord_folder_parent,tfrecord_val_folder))
sorted_names_val = natsorted(os.listdir(tfrecord_path_val))
trainfilenames_val = []
for i in sorted_names_val:
    trainfilenames_val.append(os.path.normpath(os.path.join(tfrecord_path_val,i)))
filenames_val = tf.placeholder(tf.string, shape=[None])
dataset_val = tf.data.TFRecordDataset(filenames_val)
dataset_val = dataset_val.map(_parse_function)  # Parse the record into tensors.
dataset_val = dataset_val.repeat()  # Repeat the input indefinitely.
dataset_val = dataset_val.batch(128)
iterator_val = dataset_val.make_initializable_iterator()

epochs_num = 50
steps = len_data[0] // batch_size
val_steps = val_len[0] // batch_size
# You can feed the initializer with the appropriate filenames for the current
# phase of execution, e.g. training vs. validation.
# next_elem = iterator_val.get_next()
# Initialize `iterator` with training data.

if not os.path.exists(os.path.join(Data_path,"checkpoints",ckpt_folder)):
    os.makedirs(os.path.join(Data_path,"checkpoints",ckpt_folder))

print(datetime.datetime.now())
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(iterator.initializer, feed_dict={filenames: trainfilenames})
    sess.run(iterator_val.initializer, feed_dict={filenames_val: trainfilenames_val})
    print("initialized")
    checkpoint_path = os.path.normpath(os.path.join(Data_path,"checkpoints",ckpt_folder,"weights.{epoch:02d}.hdf5"))
    checkpoint_dir = os.path.dirname(checkpoint_path)

    cp_callback = tf.keras.callbacks.ModelCheckpoint(
        checkpoint_path, verbose=1, save_best_only=True, save_weights_only=True)
        # Save weights, every 5-epochs.
#         period=1)
    early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)
    opt = tf.keras.optimizers.Adamax()
#     opt = tf.train.AdamOptimizer()
#     opt = tf.keras.optimizers.SGD()
    model.compile(loss='mean_squared_error', optimizer=opt)
    history = model.fit( iterator, steps_per_epoch=steps,epochs=epochs_num, callbacks = [cp_callback,early_stop], verbose=1,validation_data=iterator_val,validation_steps=val_steps)
#     model.save(os.path.normpath(os.path.join(Data_path, 'models', "model_3h_dataset.h5")))
#     tf.keras.models.save_model(model, os.path.normpath(os.path.join(Data_path, 'models', "model_3h_dataset.h5")))
#     model.save_weights(os.path.normpath(os.path.join(Data_path, 'models', "model_3h_dataset.h5")))
    model_json = model.to_json()
    with open(os.path.normpath(os.path.join(Data_path, 'models', "model_3.json")), "w") as json_file:
        json_file.write(model_json)
    # # serialize weights to HDF5
    model.save_weights(os.path.normpath(os.path.join(Data_path, 'models', "model_3.h5")))
    print("Saved model to disk")
    
print(datetime.datetime.now())
%matplotlib inline
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()
plt.savefig(os.path.normpath(os.path.join(Data_path,'images',ckpt_folder+'.png')))
# model_json = model.to_json()
# with open("model_10h_dataset.json", "w") as json_file:
#     json_file.write(model_json)
# model.save_weights("model_10h_dataset.h5")
# print("Saved model to disk")


imported
i was here
i was here
2019-06-20 17:06:13.576013
initialized
Epoch 1/50
774/781 [============================>.] - ETA: 0s - loss: 1.7231
Epoch 00001: val_loss improved from inf to 1.40489, saving model to /home/aut_speech/damirchi/checkpoints/3/weights.01.hdf5
781/781 [==============================] - 6s 7ms/step - loss: 1.7176 - val_loss: 1.4049
Epoch 2/50
337/781 [===========>..................] - ETA: 1s - loss: 1.2889

KeyboardInterrupt: 

In [6]:
plt.savefig(os.path.normpath(os.path.join(Data_path,'images',ckpt_folder+'.png')))

<Figure size 432x288 with 0 Axes>

In [6]:
    model_json = model.to_json()
    with open(os.path.normpath(os.path.join(Data_path, 'models', "model_3h_dataset_5.json")), "w") as json_file:
        json_file.write(model_json)
    # # serialize weights to HDF5
    model.save_weights(os.path.normpath(os.path.join(Data_path, 'models', "model_3h_dataset_5.h5")))
    print("Saved model to disk")

FailedPreconditionError: Error while reading resource variable dense_6/kernel from Container: localhost. This could mean that the variable was uninitialized. Not found: Container localhost does not exist. (Could not find resource: localhost/dense_6/kernel)
	 [[node dense_6/kernel/Read/ReadVariableOp (defined at <ipython-input-5-a00c413db055>:103) ]]
	 [[node dense_6/bias/Read/ReadVariableOp (defined at <ipython-input-5-a00c413db055>:103) ]]

Caused by op 'dense_6/kernel/Read/ReadVariableOp', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/aut_speech/damirchi_env/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/aut_speech/damirchi_env/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/aut_sp

In [3]:
file.close()

In [35]:
testload=tf.keras.models.load_model('model_10h_dataset.h5')

ValueError: No model found in config file.

In [27]:
a=tf.constant([1,2,3])
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    label_numpy = a.eval()

In [97]:
import h5py 
import tensorflow as tf
hh = h5py.File('ftr_refrmd_10h.hdf5', 'r')
d=hh['ftr_refrmd_10h'][0]
len_data=d.shape
hh.close()
len_data

(771,)

In [24]:
file.close()

In [92]:
dataset

<PrefetchDataset shapes: ((?, 257), (?,)), types: (tf.float32, tf.float32)>